<a href="https://colab.research.google.com/github/thiagofuruchima/disaster_message_classification/blob/main/models/ML%20Pipeline%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle

import nltk

In [2]:
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DISASTER.db')
df = pd.read_sql_table("CLEAN_MESSAGES", engine)
X = df['message']
Y = df.iloc[:,4:]
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
# create the NLP ML Pipeline
pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf',  MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split the data in train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [7]:
# Fit the pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
# Predict using the test data
Y_pred = pipeline.predict(X_test)

In [9]:
# Print the classification report for for each column
for i, column in enumerate(Y_train.columns):
    print("Columns: ", column)
    print(classification_report(Y_test.values[:,i], Y_pred[:,i]))
    print()

Columns:  related
              precision    recall  f1-score   support

           0       0.76      0.26      0.39      1539
           1       0.80      0.98      0.88      4967
           2       1.00      0.04      0.08        48

    accuracy                           0.80      6554
   macro avg       0.86      0.43      0.45      6554
weighted avg       0.80      0.80      0.76      6554


Columns:  request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5442
           1       0.89      0.40      0.56      1112

    accuracy                           0.89      6554
   macro avg       0.89      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554


Columns:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6332
           1       1.00      0.01      0.03       222

    accuracy                           0.97      6554
   macro avg       0.98      0.51      0.50      6554
weighted avg       0.97      0.97      0.95      6554


Columns:  death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6257
           1       0.88      0.12      0.21       297

    accuracy                           0.96      6554
   macro avg       0.92      0.56      0.60      6554
weighted avg       0.96      0.96      0.94      6554


Columns:  other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5696
           1       0.65      0.02      0.03       858

    accuracy                           0.87      6554
   macro avg       0.76      0.51      0.48      6554
weighted avg       0.84      0.87   

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Define GridSearch parameters
parameters = {'clf__estimator__n_estimators': range(100,200,100),
              'clf__estimator__min_samples_split': range(2,3)}

# Instantiate GridSearch object
cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4)

# Use GridSearch to find the best parameters
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
# Predict using the trained model with the best parameters
Y_pred = cv.predict(X_test)

In [12]:
# Print the classification report for for each column
for i, column in enumerate(Y_train.columns):
    print("Columns: ", column)
    print(classification_report(Y_test.values[:,i], Y_pred[:,i]))
    print()

Columns:  related
              precision    recall  f1-score   support

           0       0.77      0.25      0.38      1539
           1       0.80      0.98      0.88      4967
           2       1.00      0.04      0.08        48

    accuracy                           0.80      6554
   macro avg       0.86      0.42      0.45      6554
weighted avg       0.80      0.80      0.76      6554


Columns:  request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5442
           1       0.88      0.42      0.57      1112

    accuracy                           0.89      6554
   macro avg       0.89      0.70      0.75      6554
weighted avg       0.89      0.89      0.88      6554


Columns:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6332
           1       0.75      0.01      0.03       222

    accuracy                           0.97      6554
   macro avg       0.86      0.51      0.50      6554
weighted avg       0.96      0.97      0.95      6554


Columns:  death
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6257
           1       0.91      0.13      0.23       297

    accuracy                           0.96      6554
   macro avg       0.93      0.57      0.61      6554
weighted avg       0.96      0.96      0.95      6554


Columns:  other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5696
           1       0.65      0.01      0.03       858

    accuracy                           0.87      6554
   macro avg       0.76      0.51      0.48      6554
weighted avg       0.84      0.87   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [13]:
# TODO: Model is taking too long to fit
# I have to find a better engine to process it, 
# before testing new ideas

### 9. Export your model as a pickle file

In [14]:
# Save the model to pickl file
with open("DISASSTER_MODEL.pkl", 'wb') as file:
    file.write(pickle.dumps(cv))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.